# Naver news crawling

In [13]:
import sqlite3
conn = sqlite3.connect('news.db')
cur = conn.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS news;
    CREATE TABLE news(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        url TEXT NOT NULL,
        content TEXT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
conn.commit()

In [14]:
from crawler import load_driver
driver = load_driver(headless=False)

In [15]:
from bs4 import BeautifulSoup
import requests

In [16]:
url = 'https://news.naver.com'
driver.get(url)
dom = BeautifulSoup(driver.page_source, 'lxml')
newsList = [
    requests.compat.urljoin(url, tag['href'])
    for tag in dom.select('.main_component.droppable ul li a')
]

In [18]:
for news in newsList:
    driver.get(news)
    domNews = BeautifulSoup(driver.page_source, 'lxml')
    newsContents = domNews.select('#articleBodyContents')

    if newsContents:
        cur.execute(
            """
            INSERT INTO news(url, content) VALUES(?, ?);
        """, (news, newsContents[0].text.strip()))
        conn.commit()

    # cur.execute("""
    #        INSERT INTO news(url, content) VALUES(?, ?);
    #    """, (news, driver.find_element_by_css_selector('#articleBodyContents').text.strip()))
    # conn.commit()

In [19]:
cur.close()
conn.close()
driver.close()

---

In [20]:
url = 'https://mail.naver.com/'
driver = load_driver(headless=False)
# driver.implicitly_wait(3)
driver.get(url)

In [29]:
driver.find_element_by_css_selector('a#gnb_logout_button').click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


In [40]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementNotVisibleException

In [41]:
wdw = WebDriverWait(driver, 10, 0.5, (ElementNotVisibleException))

In [42]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec

In [48]:
wdw.until(
    ec.visibility_of_element_located(
        (By.CSS_SELECTOR, 'p.gnb_account + a.gnb_btn_login')), 'ERROR')
driver.find_element_by_css_selector('p.gnb_account + a.gnb_btn_login').click()

TimeoutException: Message: ERROR


In [2]:
from crawler import load_driver

In [3]:
url = 'https://mail.naver.com/'
driver = load_driver(headless=False)
# driver.implicitly_wait(3)
driver.get(url)

In [14]:
driver.find_element_by_css_selector('a.gnb_btn_login').click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


In [15]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementNotVisibleException

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec

In [16]:
wdw = WebDriverWait(driver, 10, 0.5, (ElementNotInteractableException))

In [17]:
wdw.until(
    ec.visibility_of_element_located(
        (By.CSS_SELECTOR, 'p.gnb_account + a.gnb_btn_login')), 'ERROR')
driver.find_element_by_css_selector('p.gnb_account + a.gnb_btn_login').click()

TimeoutException: Message: ERROR


In [22]:
btn = driver.find_element_by_css_selector('a.gnb_btn_login')

In [23]:
btn.is_displayed()

False

In [42]:
driver.find_element_by_id('gnb_my_namebox').click()

In [26]:
btn = driver.find_element_by_css_selector('span.gnb_txt')

In [27]:
btn.is_displayed()

False

In [43]:
btn = driver.find_element_by_css_selector('span.gnb_txt')

In [49]:
driver.find_elements_by_css_selector('span.gnb_txt')[1].click()

---

In [58]:
import sqlite3
conn = sqlite3.connect('news.db')
cur = conn.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS news;
    CREATE TABLE news(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        url TEXT NOT NULL,
        content TEXT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
conn.commit()

In [54]:
for _ in driver.find_elements_by_css_selector(
    ".hdline_news .hdline_article_tit > a, .com_list li > a"):
    if len(_.get_attribute("href")) > 1:
        cur.execute("INSERT INTO news(url, title) VALUES(?, ?);",
                    (_.get_attribute("href"), _.text))
        conn.commit()

In [60]:
driver.get("https://news.naver.com")
N = len([ _ for _ in driver.find_elements_by_css_selector(".hdline_news .hdline_article_tit > a, .com_list li > a")
         if len(_.get_attribute("href")) > 1])
for i in range(N):
    driver.find_element_by_css_selector("#lnb > ul > li:first-child > a").click()
    news = [_ for _ in 
           driver.find_elements_by_css_selector(".hdline_news .hdline_article_tit > a, .com_list li > a")
           if len (_.get_attribute("href")) > 1][i]
    news.click()
    cur.execute("insert into news(url, content) values(?,?)",
               (driver.current_url, driver.find_element_by_css_selector("#articleBodyContents").text))
    conn.commit()

---

In [71]:
driver.get('https://www.naver.com/')

In [72]:
iframe = driver.find_element_by_id('da_iframe_time')

In [73]:
driver.switch_to_frame(iframe)

C:\Users\u\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [75]:
img = driver.find_element_by_tag_name('img')

In [77]:
img.get_attribute('src')

'https://ssl.pstatic.net/tveta/libs/1246/1246137/ab2b388f2fb7985bdaaf_20190708181910995.png'

In [84]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

원본 DOM으로 switching

In [85]:
driver.switch_to_default_content()

C:\Users\u\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [97]:
from crawler import download
from bs4 import BeautifulSoup
import base64

In [151]:
driver.get('https://www.google.com/search?q=%EC%95%84%EC%9D%B4%EB%A6%B0&source=lnms&tbm=isch&sa=X&ved=0ahUKEwiwjPWKv7vjAhUKjLwKHUfMA78Q_AUIECgB&biw=767&bih=700')
for i in range(10):
    driver.implicitly_wait(1)
    
    try:
        btnNext = driver.find_element_by_id('smb')
        if btnNext and btnNext.get_property('value') == '결과 더보기':
            btnNext.click()
    except:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    
dom = BeautifulSoup(driver.page_source, 'lxml')
imgs = [tag['src'] for tag in dom.select('img.rg_ic.rg_i') if tag.has_attr('src')]

for i, img in enumerate(imgs):
    if 'base64' in img:
        content = base64.b64decode(imgs[0].split('base64,')[-1])
        ext = '.' + img.split(';')[0].split('/')[-1]
    else:
        html = download('get', img)
        content = html.content
        ext = '.' + html.headers['Content-Type'].split('/')[-1]
    
    with open('아이린/'+str(i)+ext, 'wb') as file:
        file.write(content)